In [28]:
!pip install xgboost
!pip install --upgrade setuptools
!pip install pycld2-0.41-cp39-cp39-win_amd64.whl

Processing c:\users\alvin\desktop\ir_pro_jolene\pycld2-0.41-cp39-cp39-win_amd64.whl


In [29]:
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
import time
#from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [30]:
!pip install nlpaug

In [31]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [32]:
!pip install textattack

  Using cached textattack-0.3.8-py3-none-any.whl (418 kB)
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
  Using cached lru_dict-1.1.8-cp39-cp39-win_amd64.whl (12 kB)
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
  Using cached num2words-0.5.12-py3-none-any.whl (125 kB)
  Using cached pinyin-0.4.0-py3-none-any.whl
  Using cached editdistance-0.6.2-cp39-cp39-win_amd64.whl (22 kB)
  Using cached lemminflect-0.2.3-py3-none-any.whl (769 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached OpenHowNet-2.0-py3-none-any.whl (18 kB)
  Using cached flair-0.12-py3-none-any.whl (374 kB)
  Using cached jieba-0.42.1-py3-none-any.whl
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached language_tool_python-2.7.1-py3-none-any.whl (34 kB)
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached multiprocess-0.70.14-py39-none-any.whl (132 kB)
  Usi

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 5.0.2 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [33]:
from textattack.augmentation import EmbeddingAugmenter

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [34]:
# from google.colab import drive

# drive.mount('/content/drive')

In [35]:
df = pd.read_csv('label_dataset.csv', lineterminator='\n')

df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,Polarity (Reviewer 1),Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,pos,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,pos,pos\r


In [36]:
df.rename(columns={'Polarity (Reviewer 1)':'label'}, inplace=True)

df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,label,Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,pos,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,pos,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,pos,pos\r


In [37]:
df['label'] = df['label'].replace('neg',0)

In [38]:
df['label'] = df['label'].replace('pos',1)

In [39]:
df['label'] = df['label'].replace('neu',2)

In [40]:
df.head()

,Datetime,Quarter,Likes,NFT,Text,Clean Text,vader,textblob,label,Polarity (Reviewer 2)\r
0,2022-06-27,22,2,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#27894...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
1,2023-02-11,31,2,Mutant Ape Yacht Club,🐳1 Mutant Ape Yacht Club bought for Ξ15.377\n\...,mutant ape yacht club bought floor h chg floor...,neu,neu,1.0,pos\r
2,2022-12-30,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#77 so...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
3,2022-12-29,24,0,Mutant Ape Yacht Club,🧪 Mutant Ape Yacht Club | #MAYC #NFT\n\n#23452...,mutant ape yacht club mayc nft sold eth k,neu,neu,1.0,pos\r
4,2022-12-29,24,1,Mutant Ape Yacht Club,Mutant Ape Yacht Club #27908 sold for 17 ETH (...,mutant ape yacht club sold eth nft collection ...,neu,neu,1.0,pos\r


In [41]:
df['label'].value_counts()

1.0    1053
2.0     453
0.0      95
Name: label, dtype: int64

In [42]:
df = df[['Clean Text','label']]

df.head()

,Clean Text,label
0,mutant ape yacht club mayc nft sold eth k,1.0
1,mutant ape yacht club bought floor h chg floor...,1.0
2,mutant ape yacht club mayc nft sold eth k,1.0
3,mutant ape yacht club mayc nft sold eth k,1.0
4,mutant ape yacht club sold eth nft collection ...,1.0


In [43]:
df.isnull().values.any()

True

In [44]:
df = df.dropna()

In [45]:
df.isnull().values.any()

False

In [46]:
X_train, X_test, y_train, y_test = train_test_split(df['Clean Text'], df['label'], test_size=0.2,random_state=42)

In [47]:
y_train.value_counts()

1.0    848
2.0    353
0.0     79
Name: label, dtype: int64

In [48]:
vectorizer = TfidfVectorizer(min_df = 1,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [49]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [50]:
embed_aug = EmbeddingAugmenter()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [01:16<00:00, 6.25MB/s]   
textattack: Unzipping file C:\Users\Alvin\.cache\textattack\tmpkya2pm6n.zip to C:\Users\Alvin/.cache/textattack\word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [51]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps3=embed_aug.augment(X_train[i])
    for sent in temps3:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [52]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1359,)
(1359,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_2912\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_2912\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [53]:
y_train.value_counts()

1.0    848
2.0    353
0.0    158
dtype: int64

In [54]:
aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Alvin\AppData\Roaming\nltk_data...


In [55]:
augmented_sentences=[]
augmented_sentences_labels=[]
for i in X_train.index:
  if y_train[i]==0:
    temps1=aug.augment(X_train[i],n=3)
    for sent in temps1:
      augmented_sentences.append(sent)
      augmented_sentences_labels.append(0)

In [56]:
X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)

print(X_train.shape)
print(y_train.shape)

(1833,)
(1833,)


C:\Users\Alvin\AppData\Local\Temp\ipykernel_2912\31741503.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train=X_train.append(pd.Series(augmented_sentences),ignore_index=True)
C:\Users\Alvin\AppData\Local\Temp\ipykernel_2912\31741503.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train=y_train.append(pd.Series(augmented_sentences_labels),ignore_index=True)


In [57]:
y_train.value_counts()

1.0    848
0.0    632
2.0    353
dtype: int64

In [58]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized=vectorizer.transform(X_test)

In [59]:
#XGB_classifier = GradientBoostingClassifier()
XGB_classifier = XGBClassifier()

XGB_classifier.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [60]:
t0 = time.time()
#XGB_classifier = GradientBoostingClassifier().fit(X_train_vectorized,y_train)
XGB_classifier = XGBClassifier(seed=0).fit(X_train_vectorized,y_train)
t1 = time.time()

time_linear_train = t1-t0

0.22999835014343262

In [61]:
y_pred=XGB_classifier.predict(X_test_vectorized)

In [62]:
print("Training time: %fs" % (time_linear_train))

Training time: 0.229998s


In [63]:
print(accuracy_score(y_test, y_pred))

0.822429906542056


In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.60      0.56      0.58        16
         1.0       0.86      0.91      0.88       205
         2.0       0.78      0.69      0.73       100

    accuracy                           0.82       321
   macro avg       0.74      0.72      0.73       321
weighted avg       0.82      0.82      0.82       321



In [65]:
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

In [66]:
# Define the search space
param_grid = { 
    "learning_rate": [0.1, 0.01],
    "max_depth": [3, 5, 7, 9],
    "n_estimators": [50, 200, 300, 400, 500],
    "min_child_weight": [3, 5, 7, 9],
    "subsample": [0.3, 0.5, 0.8],
    "colsample_bytree": [0.3, 0.5 , 0.8]
    }
# Set up score
scoring = ['f1']
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [67]:
# Define grid search
grid_search = GridSearchCV(estimator=XGB_classifier, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           refit='f1', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)
# Fit grid search
grid_result = grid_search.fit(X_train_vectorized,y_train)
# Print grid search summary
grid_result
# Print the best score and the corresponding hyperparameters
print(f'The best score is {grid_result.best_score_:.4f}')
#print('The best score standard deviation is', round(grid_result.cv_results_['std_test_recall'][grid_result.best_index_], 4))
print(f'The best hyperparameters are {grid_result.best_params_}')

C:\Users\Alvin\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


The best score is nan
The best hyperparameters are {'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.3}


In [68]:
# Make prediction using the best model
grid_predict = grid_search.predict(X_test_vectorized)
# Get predicted probabilities
#grid_predict_prob = grid_search.predict_proba(X_test_vectorized)[:,1]

In [69]:
print(accuracy_score(y_test, y_pred))

0.822429906542056


In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.60      0.56      0.58        16
         1.0       0.86      0.91      0.88       205
         2.0       0.78      0.69      0.73       100

    accuracy                           0.82       321
   macro avg       0.74      0.72      0.73       321
weighted avg       0.82      0.82      0.82       321

